<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/Whisper%20on%20google%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Whisper on google colab**

https://qiita.com/walnut-pro/items/0124a5a0c83c9b4e2669

https://qiita.com/walnut-pro/items/69864b0a074bd773711f

In [1]:
! pip install git+https://github.com/openai/whisper.git --q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.8 MB/s eta 0:00:00


In [5]:
!pip install whisper
import whisper
model = whisper.load_model("medium")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


100%|█████████████████████████████████████| 1.42G/1.42G [00:29<00:00, 52.0MiB/s]


In [6]:
# confirm device 
model.device #device(type='cuda', index=0)

device(type='cuda', index=0)

# **保存用フォルダ作成**

In [7]:
import os

# Add directory into content folder
checkDownLoadFolder = os.path.exists("download")
if not checkDownLoadFolder:
  os.mkdir("download")

# **YouTubeから音声をDL**

In [8]:
!pip install youtube-dl --q
!pip install yt-dlp --q
!pip install ffmpeg --q
#!python -m pip install -U yt-dlp

#from __future__ import unicode_literals
import youtube_dl
import pandas as pd
from yt_dlp import YoutubeDL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [9]:
ydl_opts = {
    'format': 'best',
    'outtmpl': f"orig.mp4",  # name the location
}



with YoutubeDL(ydl_opts) as ydl:
    url_list = ["https://www.youtube.com/watch?v=N3vAAynw0G0"] #リストを直接入力する場合
    #url_list = df.iloc[:, 0] #CSV
    ydl.download(url_list)


[youtube] Extracting URL: https://www.youtube.com/watch?v=N3vAAynw0G0
[youtube] N3vAAynw0G0: Downloading webpage
[youtube] N3vAAynw0G0: Downloading android player API JSON
[info] N3vAAynw0G0: Downloading 1 format(s): 22
[download] Destination: orig.mp4
[download] 100% of  123.33MiB in 00:00:03 at 34.48MiB/s  


# **文章を15分毎に分割する (GPTに入力しやすくするため）**





In [16]:
# mp4 --> mp3
import ffmpeg 
 
# 入力 
stream = ffmpeg.input("orig.mp4") 
# 出力 
stream = ffmpeg.output(stream, "orig.mp3") 
# 実行 
ffmpeg.run(stream)

(None, None)

# **文字起こし**

In [22]:
fileName = "orig.mp3"

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(f"/content/{fileName}")

outputTextsArr = []
while audio.size > 0:
  tirmedAudio = whisper.pad_or_trim(audio)
  # trimedArray.append(tirmedAudio)
  startIdx = tirmedAudio.size
  audio = audio[startIdx:]

  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(tirmedAudio).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  # print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  options = whisper.DecodingOptions()
  result = whisper.decode(model, mel, options)

  # print the recognized text
  outputTextsArr.append(result.text)
  print(result.text)

outputTexts = ' '.join(outputTextsArr)
print(outputTexts)

# Write into a text file
with open(f"download/{fileName.split('.')[0]}.txt", "w") as f:
  f.write(f"▼ Transcription of {fileName.split('.')[0]}\n")
  f.write(outputTexts)

ミラクルの大学恋愛順です。今回は男女の恋愛感の違い5点ということで、男女では実は恋愛感、恋愛の考え方、そして恋愛の本能です。これが全く違うわけです。皆さん、これ超重要なので覚えておいてください。これ本当に実感できればもうモテ男まっしぐらです。女性に困ることはなくなります。女性から寄ってくるようなめちゃめちゃカッコいい男になれる超重要ポイントです。
真逆です。これがね、僕はね、理解するのにね、もうほんとね、4、5年かかりましたけども男女ってシステムが真逆なんですよ。生物構造上ね。だってさ、男がさ、出てる側でさ、女の子が凹んでる側じゃないですか。なんとなくわかる。で、その構造を元に我々の本能っていうのは作られてるわけですよ。ってなった時に、基本的に性質は真逆です。真逆の性質なんですよ。
女の子はこうやったら嬉しいっていうのと女の子はこうやったら嬉しいっていうのが全然違うんですよということですじゃあよじゃあよじゃあよこれさこれさ知らないとできなくない?そうなんですよ知らないとね勉強しないとできないんですよじゃあ女の子はどうやったら好きになるのかどうやったら愛情を感じるのかとかねどんなプレゼントを渡したら喜ぶのかとかどんな関係性を望んでるんですかどうすれば浮気にするんですか
とかねみんな気になるよねそうなんですよけどねこれさ われわれの感覚でやきっとこうやったら女の子って喜んでくれるに違いないと思ってやるでしょもうまとはずれも花はだしそして女性からはこう思われますああ こいつわかってねーなとね悲しいよねせっかく女の子を喜ばせようと思ってね幸せにしようと思ってやってることは知識不足によってもう水の泡ですすべてが水の泡 残念でしたってなるわけですなので今回はですね男性の恋愛感の違い
ポイント5つバッチリお伝えします これ全部見て理解していただけたら女性を喜ばせる方法女性を好きにさせる方法そして付き合った女性とずっといい関係を続けていく方法なんだろうね 嫁さんになってから結婚してからもずっといい関係を築いていこう方だったりとかもちろん恋愛関係だけじゃなくて会社の中のね女性とか女友達とかね同じクラス同じサークルです という女の子からもこの人わかってるなとこの人モテるなこの人素敵だなって思わ
いいことばっかりですのでぜひ全部抑えてそして女心が分かる最強のモテ男になって いきましょうで

# **Summarize on ChatGPT**

In [32]:
from google.colab import drive
drive.mount("/content/drive")

!pip install openai --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
openai.api_key = key[3]
deepl_auth_key = key[1]

translator = deepl.Translator(deepl_auth_key)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# テキストファイルを読み込み
text_path = "/content/drive/MyDrive/secondJob/Transcription/youshu_whisper_medium.txt"
with open(text_path, 'r') as f:
    content = f.read()  # すべての行を読み込む
print(content)

▼ Transcription of orig.mp3
ミラクルの大学恋愛順です。今回は男女の恋愛感の違い5点ということで、男女では実は恋愛感、恋愛の考え方、そして恋愛の本能です。これが全く違うわけです。皆さん、これ超重要なので覚えておいてください。これ本当に実感できればもうモテ男まっしぐらです。女性に困ることはなくなります。女性から寄ってくるようなめちゃめちゃカッコいい男になれる超重要ポイントです。 真逆です。これがね、僕はね、理解するのにね、もうほんとね、4、5年かかりましたけども男女ってシステムが真逆なんですよ。生物構造上ね。だってさ、男がさ、出てる側でさ、女の子が凹んでる側じゃないですか。なんとなくわかる。で、その構造を元に我々の本能っていうのは作られてるわけですよ。ってなった時に、基本的に性質は真逆です。真逆の性質なんですよ。 女の子はこうやったら嬉しいっていうのと女の子はこうやったら嬉しいっていうのが全然違うんですよということですじゃあよじゃあよじゃあよこれさこれさ知らないとできなくない?そうなんですよ知らないとね勉強しないとできないんですよじゃあ女の子はどうやったら好きになるのかどうやったら愛情を感じるのかとかねどんなプレゼントを渡したら喜ぶのかとかどんな関係性を望んでるんですかどうすれば浮気にするんですか とかねみんな気になるよねそうなんですよけどねこれさ われわれの感覚でやきっとこうやったら女の子って喜んでくれるに違いないと思ってやるでしょもうまとはずれも花はだしそして女性からはこう思われますああ こいつわかってねーなとね悲しいよねせっかく女の子を喜ばせようと思ってね幸せにしようと思ってやってることは知識不足によってもう水の泡ですすべてが水の泡 残念でしたってなるわけですなので今回はですね男性の恋愛感の違い ポイント5つバッチリお伝えします これ全部見て理解していただけたら女性を喜ばせる方法女性を好きにさせる方法そして付き合った女性とずっといい関係を続けていく方法なんだろうね 嫁さんになってから結婚してからもずっといい関係を築いていこう方だったりとかもちろん恋愛関係だけじゃなくて会社の中のね女性とか女友達とかね同じクラス同じサークルです という女の子からもこの人わかってるなとこの人モテるなこの人素敵だなって思わ いいことばっかりですのでぜひ全部抑え

In [45]:
def split_string(string):
    # 文字列を1000字ごとに分割する
    return [string[i:i+1000] for i in range(0, len(string), 1000)]

content_split = split_string(content)
len(content_split)

11

In [47]:
def chatGPT(prompt):
    #モデルを指定
    model_engine = "text-davinci-003"

    #指示（Prompt）を設定
    prompt = prompt

    #推論を実行
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    #推論結果を出力
    message = completions.choices[0].text
    return message

In [58]:
revised = []

for text in content_split:

    reply = chatGPT(f"""
    あなたはプロのスピーチ原稿ライターです。次の文章は会議における著名な教授の講演原稿草案です。日本語で句読点をつけて書き起こしてください。文脈が合わない箇所については語句を推測して修正してください。
    内容は以下の通りです。
    {text}
    """)

    revised.append(reply)

    print(revised)

print(revised)

['\nミラクルの大学恋愛順です。今回は男女の恋愛感の違い5点ということで、男女では実は恋愛感、恋愛の考え方、そして恋愛の本能です。これが全く違うわけです。皆さん、これ超重要なので覚えておいてください。これ本当に実感できればもうモテ男まっしぐらです。女性に困ることはなくなります。女性から寄ってくるようなめちゃめちゃカッコいい男になれる超重要ポイントです。\n\n真逆です。これがね、僕はね、理解するのにね、もうほんとね、4、5年かかりましたけども男女ってシステムが真逆なんですよ。生物構造上ね。だってさ、男がさ、出てる側でさ、女の子が凹んでる側じゃないですか。なんとなくわかる。で、その構造を元に我々の本能っていうのは作られてるわけですよ。\n\nってなった時に、基本的に性質は真逆です。真逆の性質なんですよ。女の子はこうやったら嬉しいっていうのと女の子はこうやったら嬉しいっていうのが全然違うんですよということですじゃあよじゃあよじゃあよこれさこれさ知らないとできなくない?そうなんですよ知らないとね勉強しないとできないんですよじゃあ女の子はどうやったら好きになるのかどうやったら愛情を感じるのかとかねどんなプレゼントを渡したら喜ぶのかとかどんな関係性を望んでるんですかどうすれば浮気にするんですか とかねみんな気になるよねそうなんですよ。\n\nけどね、これさ、われわれの感覚でやきっとこうやったら女の子って喜んでくれるに違いないと思ってやるでしょもうまとはずれも花はだしそして女性からはこう思われますああ こいつわかってねーなとね悲しいよね。せっかく女の子を喜ばせようと思ってね幸せにしようと思ってやってることは知識不足によってもう水の泡です。すべてが水の泡 残念でしたってなるわけです。\n\nなので今回は、ですね、男性の恋愛感の違い、', '\n    てそして女心が分かる最強のモテ男になっていきましょう。では行ってみましょう。さあ、では順番に行きましょう。1つ目は行為の最大値です。あの生物的に本能的にね、残念だから決まっております。男は抱く前が一番可愛く見えます。はい。で女性の場合は、やった後が一番可愛く見えます。なのかっていうのを理解しておくのが大事です。でそしたら、どういうコミュニケーションを女性と取ればいいのかがわかります。で、なんでなのかっていうことなんですが、まず大前

In [67]:
# 作成したreviseを保存
import os

# リストの文字列を結合
text_list = revised
text = ' '.join(text_list)

# 保存先のパスを指定
save_path = '/content/drive/MyDrive/secondJob/Transcription/youshu_whisper_chatGPT_revised.txt'

# テキストファイルを作成して、文字列を書き込む
with open(save_path, 'w') as f:
    f.write(text)

# 保存が完了したことを確認する
if os.path.exists(save_path):
    print('テキストファイルが正常に保存されました。')
else:
    print('テキストファイルの保存に失敗しました。')

# リスト内の要素数を取得して表示する
num_items = len(text_list)
print(f'リスト内の要素数は {num_items} です。')


テキストファイルが正常に保存されました。
リスト内の要素数は 11 です。


In [ ]:
from IPython.display import display, Markdown, Latex

# 要約
summarized = ""
for text in revised:
    
    summarized = chatGPT(f"""

    次の文章は著名な教授の講演の書き起こしを分割したものです。
    話している内容を大見出しと小見出しをつけてまとめて下さい。
    大見出しには#を、小見出しには##をつけてMarkdown形式にして下さい。
    それぞれの小見出しの後に、それを説明する根拠となる文章を加えて下さい。

    {summarized}
    {text}

    """)

    display(Markdown(summarized))

display(Markdown(summarized))


In [77]:
from IPython.display import display, Markdown, Latex

# 要約

rewrite = chatGPT(f"""

あなたはプロのシナリオライターです。次の文章は著名な教授の講演の概要です。

それぞれの小見出しについての説明を、関連する内容を加えることにより、それぞれ1000文字程度にして下さい。
もし文字数が足りなければ、概要に載っていない内容を付け加えても大丈夫です。

概要は以下の通りです。
{summarized}

""")

display(Markdown(rewrite))

 # 男女の恋愛感の違い
     ## 男女では実は恋愛感、恋愛の考え方、そして恋愛の本能で全く違う
     生物構造上では男性と女性の本能は全く異なっており、男女の恋愛感は真逆である。
    
     ## 男女の恋愛感を理解するのには時間がかかる
     男性が女性を喜ばせようとする時、知識不足により、その行動が水の泡になってしまう可能性がある。
    
     ## 今回は、男性の恋愛感の違いについて話す
     今回の講演では、男性の恋愛感の違いについて話す。男性が女性を喜ばせるためには、男女の恋愛感を理解し、本能を正しく理解する必要がある。これは、男性が女性を守るために生まれた本能であり、男性が女性のことを大事にすることが重要であることを示している。そのため、男性は女性に対して、抱いた後から好きになるような気持ちを持ち、毎日の連絡や、自分だけが女性を愛しているという気持ちを伝える必要がある。